<a href="https://colab.research.google.com/github/FaySal-Coder/Quater-2-AI-Project/blob/main/PRO2_RAG_Project_with_Pinecone_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SETUP

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.4 MB/s eta 0:00:00


## CREATE **PINECONE** API KEY

In [ ]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

# CREATE PINECONE DATABASE INDEX

In [ ]:
import time

index_name = "rag-proj2"

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("I had chocalate chip pancakes and scrambled eggs for breakfast this morning")
vector[:5]



[0.003429398639127612,
 0.002272629411891103,
 -0.024901917204260826,
 -0.04553511366248131,
 0.049954164773225784]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]



uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)




['75fbc062-fe1d-46a7-b759-3683b1bf973c',
 '11f71adb-425e-4e26-8142-93a20af5d31a',
 '26c2b72d-8d52-4680-81c1-f6bfdc6d0563',
 'a605f506-7b64-4933-b43a-2f33274771ff',
 '4ec4c401-5598-409a-943b-5477742e13eb',
 'f71096c1-2470-4226-8f9d-42b3e5774fc1',
 '5a86f398-3c86-4acb-8912-a4f9ddc5575c',
 '403f5e2d-4c26-4965-b8c5-8fd6e0f10bcb',
 '0e6fe6f4-2db4-4163-acea-c406313411cb',
 '06911359-8dc7-4eeb-928c-7adf846610a7']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=1,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = userdata.get('GOOGLE_API_KEY')
)

In [ ]:
def answer_to_user(query: str):

 vector_results = vector_store.similarity_search(query, k=2)
 print(len(vector_results))

 final_answer = llm.invoke(f"Answer to user query: {query}, Here are some reference to answer {vector_results}")
 return final_answer


In [ ]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")

2


In [ ]:
answer.content

"LangChain simplifies working with Large Language Models (LLMs).  The provided documents offer anecdotal evidence of its use, with one user highlighting a different framework (LangGraph) and another mentioning a project built with LangChain.  Neither document directly explains LangChain's abstractions, but the second tweet implicitly confirms its use for building projects."